# 🤖 7-DOF Robot Topology Optimization using Pinocchio + CasADi

This notebook demonstrates how to optimize the link lengths of a 7-DOF robotic arm using CasADi and Pinocchio.  
The objective is to maximize the average manipulability over a set of sampled configurations.


In [ ]:
import time
import numpy as np
import casadi
from casadi import SX, vertcat, Function, nlpsol
import pinocchio as pin
from pinocchio import casadi as cpin
import matplotlib.pyplot as plt
from pinocchio.visualize import MeshcatVisualizer
import meshcat
vis = meshcat.Visualizer()


## 🧩 1. Define link length variables
We define symbolic variables for the 7 link lengths and provide default initial values.


## 🏗️ 2. Build a 7-DOF planar arm in Pinocchio
Each joint is a revolute joint around the Y-axis (JointModelRY).
Links are aligned along X and their lengths are parameters.


In [ ]:
def build_symbolic_robot(lengths_sx):
    model = cpin.Model()
    joint_placement = cpin.SE3.Identity()
    parent_id = 0

    for i in range(7):
        joint_name = f'joint{i}'
        joint_id = model.addJoint(parent_id, cpin.JointModelRY(), joint_placement, joint_name)
        inertia = cpin.Inertia.Random()
        model.appendBodyToJoint(joint_id, inertia, cpin.SE3.Identity())
        model.addFrame(cpin.Frame(f'link{i}', joint_id, 0, cpin.SE3.Identity(), cpin.FrameType.BODY))
        parent_id = joint_id

        joint_placement = cpin.SE3.Identity()
        joint_placement.translation = vertcat(lengths_sx[i], 0, 0)

    # model.addBodyFrame("ee", parent_id, cpin.SE3.Identity())
    return model



## 📐 3. Manipulability score function
This function computes the average Yoshikawa manipulability index over a set of random joint configurations.


In [ ]:
def symbolic_manipulability(model, lengths_sx):
    nq = model.nq
    q = SX.sym("q", nq)

    data = cpin.Data(model)
    cpin.forwardKinematics(model, data, q)
    J = cpin.computeJointJacobian(model, data, q, model.njoints - 1)[:3, :]
    manipulability = casadi.sqrt(casadi.det(J @ J.T) + 1e-8)
    return Function("manip_score", [q], [manipulability])



## 🔧 4. Optimization with CasADi
We define an objective function that returns the negative manipulability score for given link lengths, and solve it using IPOPT.


In [ ]:
def build_objective_function(lengths_sym):
    model = build_symbolic_robot(lengths_sym)
    manip_func = symbolic_manipulability(model, lengths_sym)
    nq = model.nq

    def objective_func(L_vals_np):
        manip_scores = []
        for _ in range(40):
            q_sample = np.random.uniform(-np.pi, np.pi, nq)
            manip_scores.append(manip_func(q_sample).full().item())
        return -np.mean(manip_scores)

    return objective_func




In [ ]:
L_sym = SX.sym("L", 7)                     # 7 symbolic link lengths
objective_np = build_objective_function(L_sym)  # builds model, computes manip

obj_func = Function("f", [L_sym], [objective_np(L_sym)])  # wrap in CasADi Function

nlp = {"x": L_sym, "f": obj_func(L_sym)}
solver = nlpsol("solver", "ipopt", nlp)

L0 = np.array([0.4] * 7)
res = solver(x0=L0, lbx=0.1, ubx=1.0)
L_opt = res["x"].full().flatten()
print("✅ Optimal link lengths:", L_opt)


## 🧑‍🎨 5. Visualize the optimized robot
We build the robot using the optimal link lengths and display it using Meshcat.


In [ ]:
import numpy as np
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf
import random

# --- Viewer
vis = meshcat.Visualizer()


# --- Optimized link lengths
L_opt = [0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1]  # Replace by your optimized result

# --- Colors: use HSV space to generate distinct colors
def hsv_to_rgb(h, s, v):
    import colorsys
    return tuple(int(255 * c) for c in colorsys.hsv_to_rgb(h, s, v))

colors = [hsv_to_rgb(i / len(L_opt), 0.8, 0.9) for i in range(len(L_opt))]

# --- Visual parameters
link_width = 0.04
link_height = 0.04

# --- Initial transform
T = np.eye(4)

for i, L in enumerate(L_opt):
    name = f"link_{i}"
    
    # Crée une boîte
    box = g.Box([L, link_width, link_height])
    color = colors[i]
    mat = g.MeshLambertMaterial(color=color[0] << 16 | color[1] << 8 | color[2])
    vis[name].set_object(box, mat)

    # Place la boîte au bon endroit (centrée sur son link)
    offset = tf.translation_matrix([L / 2, 0, 0])
    vis[name].set_transform(T @ offset)

    # Ajoute une rotation autour de Y entre -10° et +10°
    theta_deg = random.uniform(-10, 10)
    theta_rad = np.deg2rad(theta_deg)
    R_y = tf.rotation_matrix(theta_rad, [0, 1, 0])

    # Avance : translation + rotation
    T = T @ tf.translation_matrix([L, 0, 0]) @ R_y
vis.jupyter_cell()


In [ ]:
from casadi import SX, vertcat
from pinocchio import casadi as cpin

# Symbolic variables
L = SX.sym("L", 7)              # link lengths
theta = SX.sym("theta", 21)     # 7 axis-angle vectors (3D each)

# Initialize symbolic Pinocchio model
model = cpin.Model()
joint_placement = cpin.SE3.Identity()
parent_id = 0

for i in range(7):
    # Joint name
    joint_name = f"joint{i}"

    # Orientation between links (symbolic axis-angle)
    omega = theta[3*i:3*(i+1)]
    R = cpin.exp3(omega)  # Rotation matrix
    p = vertcat(L[i], 0, 0)  # Translation along local X
    T = cpin.SE3(R, p)       # Full transform for next joint

    # Add revolute Y joint at current placement
    joint_id = model.addJoint(parent_id, cpin.JointModelRY(), joint_placement, joint_name)

    # Add a dummy inertia (required)
    model.appendBodyToJoint(joint_id, cpin.Inertia.Random(), cpin.SE3.Identity())

    # Add a body frame for visualization/reference
    model.addFrame(cpin.Frame(f"link{i}", joint_id, 0, cpin.SE3.Identity(), cpin.FrameType.BODY))

    # Update for next joint
    joint_placement = T
    parent_id = joint_id

# Résumé du modèle
print(f"✅ Model built with {model.njoints} joints and {len(model.frames)} frames.")
print("Joint names:", model.names)



In [ ]:
import numpy as np
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

# === Viewer
vis = meshcat.Visualizer()


# === Données de test (remplace plus tard par sortie optimisée)
L = [0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1]
theta = np.random.uniform(-2, 2, size=(7, 3))  # axe-angle aléatoire pour chaque joint

# === Couleurs distinctes
def hsv_to_rgb(h, s, v):
    import colorsys
    r, g_, b = colorsys.hsv_to_rgb(h, s, v)
    return int(r * 255) << 16 | int(g_ * 255) << 8 | int(b * 255)

colors = [hsv_to_rgb(i / 7, 0.8, 0.9) for i in range(7)]

# === Transformée initiale
T = np.eye(4)

for i in range(7):
    name = f"link_{i}"
    length = L[i]
    omega = theta[i]

    # Boîte pour le link
    box = g.Box([length, 0.04, 0.04])
    material = g.MeshLambertMaterial(color=colors[i])
    vis[name].set_object(box, material)

    # Positionner la boîte au bon endroit (centrée)
    center_offset = tf.translation_matrix([length / 2, 0, 0])
    vis[name].set_transform(T @ center_offset)

    # Mise à jour du transform global : translation + rotation axis-angle
    T = T @ tf.translation_matrix([length, 0, 0]) @ tf.rotation_matrix(np.linalg.norm(omega) + 1e-8, omega / (np.linalg.norm(omega) + 1e-8))
vis.jupyter_cell()

In [ ]:
# 🤖 Symbolic Workspace Estimation with CasADi + Pinocchio.casadi + Optimization

import numpy as np
from casadi import SX, vertcat, Function, sum1, nlpsol
from pinocchio import casadi as cpin

# --- Step 1: Define symbolic variables
L = SX.sym("L", 7)              # Link lengths
theta = SX.sym("theta", 21)     # Orientations: 7 axis-angle vectors (3D each)
q = SX.sym("q", 7)              # Joint configuration

# --- Step 2: Build symbolic model and f_kine

def build_f_kine():
    model = cpin.Model()
    joint_placement = cpin.SE3.Identity()
    parent_id = 0

    for i in range(7):
        omega = theta[3*i:3*(i+1)]
        R = cpin.exp3(omega)
        p = vertcat(L[i], 0, 0)
        T = cpin.SE3(R, p)

        joint_id = model.addJoint(parent_id, cpin.JointModelRY(), joint_placement, f"joint{i}")
        model.appendBodyToJoint(joint_id, cpin.Inertia.Random(), cpin.SE3.Identity())
        model.addFrame(cpin.Frame(f"link{i}", joint_id, 0, cpin.SE3.Identity(), cpin.FrameType.BODY))

        joint_placement = T
        parent_id = joint_id

    data = cpin.Data(model)
    cpin.forwardKinematics(model, data, q)
    p_ee = data.oMi[model.njoints - 1].translation
    return Function("f_kine", [L, theta, q], [p_ee])

f_kine = build_f_kine()

# --- Step 3: Define workspace_metric(L, theta) by sampling N q

def build_workspace_metric(N=50):
    q_samples = np.random.uniform(-np.pi, np.pi, size=(N, 7))
    p_samples = [f_kine(L, theta, q_sample) for q_sample in q_samples]
    mean_p = sum(p_samples) / N
    spread = sum([sum1((p - mean_p)**2) for p in p_samples]) / N
    return Function("workspace_metric", [L, theta], [spread])

workspace_metric = build_workspace_metric()

# --- Step 4: Optimize workspace_metric(L, theta)
x = vertcat(L, theta)
nlp = {"x": x, "f": -workspace_metric(x[:7], x[7:])}  # negative to maximize
solver = nlpsol("solver", "ipopt", nlp)

x0 = np.concatenate([np.ones(7) * 0.3, np.zeros(21)])  # init lengths and orientations
lbx = [0.1]*7 + [-np.pi]*21
ubx = [1.0]*7 + [np.pi]*21

result = solver(x0=x0, lbx=lbx, ubx=ubx)
x_opt = result["x"].full().flatten()

L_opt = x_opt[:7]
theta_opt = x_opt[7:]

print("\n✅ Optimized link lengths:", L_opt)
print("✅ Optimized orientations (axis-angle):", theta_opt)

In [ ]:
# 🤖 Symbolic Workspace Estimation with CasADi + Pinocchio.casadi + Optimization + Meshcat Visualisation

import numpy as np
from casadi import SX, vertcat, Function, sum1, nlpsol
from pinocchio import casadi as cpin
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

# --- Step 1: Define symbolic variables
L = SX.sym("L", 7)              # Link lengths
theta = SX.sym("theta", 21)     # Orientations: 7 axis-angle vectors (3D each)
q = SX.sym("q", 7)              # Joint configuration

# --- Step 2: Build symbolic model and f_kine

def build_f_kine():
    model = cpin.Model()
    joint_placement = cpin.SE3.Identity()
    parent_id = 0

    for i in range(7):
        omega = theta[3*i:3*(i+1)]
        R = cpin.exp3(omega)
        p = vertcat(L[i], 0, 0)
        T = cpin.SE3(R, p)

        joint_id = model.addJoint(parent_id, cpin.JointModelRY(), joint_placement, f"joint{i}")
        model.appendBodyToJoint(joint_id, cpin.Inertia.Random(), cpin.SE3.Identity())
        model.addFrame(cpin.Frame(f"link{i}", joint_id, 0, cpin.SE3.Identity(), cpin.FrameType.BODY))

        joint_placement = T
        parent_id = joint_id

    data = cpin.Data(model)
    cpin.forwardKinematics(model, data, q)
    p_ee = data.oMi[model.njoints - 1].translation
    return Function("f_kine", [L, theta, q], [p_ee])

f_kine = build_f_kine()

# --- Step 3: Define workspace_metric(L, theta) by sampling N q

def build_workspace_metric(N=50):
    q_samples = np.random.uniform(-np.pi, np.pi, size=(N, 7))
    p_samples = [f_kine(L, theta, q_sample) for q_sample in q_samples]
    mean_p = sum(p_samples) / N
    spread = sum([sum1((p - mean_p)**2) for p in p_samples]) / N
    return Function("workspace_metric", [L, theta], [spread]), q_samples

workspace_metric, q_samples = build_workspace_metric()

# --- Step 4: Optimize workspace_metric(L, theta)
x = vertcat(L, theta)
nlp = {"x": x, "f": -workspace_metric(x[:7], x[7:])}  # negative to maximize
solver = nlpsol("solver", "ipopt", nlp)

x0 = np.concatenate([np.ones(7) * 0.3, np.zeros(21)])  # init lengths and orientations
lbx = [0.1]*7 + [-np.pi]*21
ubx = [1.0]*7 + [np.pi]*21

result = solver(x0=x0, lbx=lbx, ubx=ubx)
x_opt = result["x"].full().flatten()

L_opt = x_opt[:7]
theta_opt = x_opt[7:]

print("\n✅ Optimized link lengths:", L_opt)
print("✅ Optimized orientations (axis-angle):", theta_opt)

# --- Step 5: Visualize optimized robot and workspace in Meshcat
vis = meshcat.Visualizer()
vis.delete()

T = np.eye(4)
def hsv_to_rgb(h, s, v):
    import colorsys
    r, g_, b = colorsys.hsv_to_rgb(h, s, v)
    return int(r * 255) << 16 | int(g_ * 255) << 8 | int(b * 255)
colors = [hsv_to_rgb(i / 7, 0.8, 0.9) for i in range(7)]

for i in range(7):
    name = f"link_{i}"
    length = L_opt[i]
    omega = theta_opt[3*i:3*(i+1)]

    box = g.Box([length, 0.04, 0.04])
    material = g.MeshLambertMaterial(color=colors[i])
    vis[name].set_object(box, material)

    offset = tf.translation_matrix([length / 2, 0, 0])
    vis[name].set_transform(T @ offset)

    R = tf.rotation_matrix(np.linalg.norm(omega)+1e-8, omega / (np.linalg.norm(omega)+1e-8))
    T = T @ tf.translation_matrix([length, 0, 0]) @ R

# --- Display sampled workspace points
for i, q_sample in enumerate(q_samples):
    p = f_kine(L_opt, theta_opt, q_sample).full().flatten()
    vis[f"workspace/pt_{i}"].set_object(g.Sphere(0.01), g.MeshLambertMaterial(color=0xff0000))
    vis[f"workspace/pt_{i}"].set_transform(tf.translation_matrix(p))
vis.jupyter_cell()

In [ ]:
# 🤖 Symbolic Workspace Estimation with CasADi + Pinocchio.casadi + Optimization + Meshcat Visualisation (Full 3D: L, θ, Fixed Joint Types)

import numpy as np
from casadi import SX, vertcat, Function, sum1, nlpsol
from pinocchio import casadi as cpin
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

# --- Step 1: Define symbolic variables
L = SX.sym("L", 7)              # Link lengths
theta = SX.sym("theta", 21)     # Orientations: 7 axis-angle vectors (3D each)
q = SX.sym("q", 7)              # Joint configuration

# --- Step 2: Build symbolic model and f_kine with fixed joint types
# Joint types: roll, pitch, yaw, roll, yaw, roll, yaw
joint_axes = [
    SX([1.0, 0.0, 0.0]),  # roll
    SX([0.0, 1.0, 0.0]),  # pitch
    SX([0.0, 0.0, 1.0]),  # yaw
    SX([1.0, 0.0, 0.0]),  # roll
    SX([0.0, 0.0, 1.0]),  # yaw
    SX([1.0, 0.0, 0.0]),  # roll
    SX([0.0, 0.0, 1.0])   # yaw
]

def build_f_kine_fixed():
    model = cpin.Model()
    joint_placement = cpin.SE3.Identity()
    parent_id = 0

    for i in range(7):
        omega = theta[3*i:3*(i+1)]
        R = cpin.exp3(omega)
        p = vertcat(L[i], 0, 0)
        T = cpin.SE3(R, p)

        axis_i = joint_axes[i]
        joint_id = model.addJoint(parent_id, cpin.JointModelRevoluteUnaligned(axis_i), joint_placement, f"joint{i}")
        model.appendBodyToJoint(joint_id, cpin.Inertia.Random(), cpin.SE3.Identity())
        model.addFrame(cpin.Frame(f"link{i}", joint_id, 0, cpin.SE3.Identity(), cpin.FrameType.BODY))

        joint_placement = T
        parent_id = joint_id

    data = cpin.Data(model)
    cpin.forwardKinematics(model, data, q)
    p_ee = data.oMi[model.njoints - 1].translation
    return Function("f_kine_fixed", [L, theta, q], [p_ee])

f_kine = build_f_kine_fixed()

# --- Step 3: Define workspace_metric(L, theta) by sampling N q
def build_workspace_metric(N=50):
    q_samples = np.random.uniform(-np.pi, np.pi, size=(N, 7))
    p_samples = [f_kine(L, theta, q_sample) for q_sample in q_samples]
    mean_p = sum(p_samples) / N
    spread = sum([sum1((p - mean_p)**2) for p in p_samples]) / N
    return Function("workspace_metric", [L, theta], [spread]), q_samples

workspace_metric, q_samples = build_workspace_metric()

# --- Step 4: Optimize workspace_metric(L, theta)
x = vertcat(L, theta)
nlp = {"x": x, "f": -workspace_metric(x[:7], x[7:])}  # negative to maximize
solver = nlpsol("solver", "ipopt", nlp)

x0 = np.concatenate([np.ones(7) * 0.3, np.zeros(21)])
lbx = [0.1]*7 + [-np.pi]*21
ubx = [0.5]*7 + [np.pi]*21

result = solver(x0=x0, lbx=lbx, ubx=ubx)
x_opt = result["x"].full().flatten()

L_opt = x_opt[:7]
theta_opt = x_opt[7:]

print("\n✅ Optimized link lengths:", L_opt)
print("✅ Optimized orientations (axis-angle):", theta_opt.reshape(7, 3))

# --- Step 5: Visualize optimized robot and workspace in Meshcat
vis = meshcat.Visualizer()

T = np.eye(4)
def hsv_to_rgb(h, s, v):
    import colorsys
    r, g_, b = colorsys.hsv_to_rgb(h, s, v)
    return int(r * 255) << 16 | int(g_ * 255) << 8 | int(b * 255)
colors = [hsv_to_rgb(i / 7, 0.8, 0.9) for i in range(7)]

for i in range(7):
    name = f"link_{i}"
    length = L_opt[i]
    omega = theta_opt[3*i:3*(i+1)]

    box = g.Box([length, 0.04, 0.04])
    material = g.MeshLambertMaterial(color=colors[i])
    vis[name].set_object(box, material)

    offset = tf.translation_matrix([length / 2, 0, 0])
    vis[name].set_transform(T @ offset)

    R = tf.rotation_matrix(np.linalg.norm(omega)+1e-8, omega / (np.linalg.norm(omega)+1e-8))
    T = T @ tf.translation_matrix([length, 0, 0]) @ R

# --- Display sampled workspace points
for i, q_sample in enumerate(q_samples):
    p = f_kine(L_opt, theta_opt, q_sample).full().flatten()
    vis[f"workspace/pt_{i}"].set_object(g.Sphere(0.01), g.MeshLambertMaterial(color=0xff0000))
    vis[f"workspace/pt_{i}"].set_transform(tf.translation_matrix(p))
vis.jupyter_cell()

In [ ]:
# 🤖 Symbolic Workspace Estimation with CasADi + Pinocchio.casadi + Optimization + Meshcat Visualisation (Full 3D: L, θ, Fixed Joint Types, Variable Y Offset)

import numpy as np
from casadi import SX, vertcat, Function, sum1, nlpsol
from pinocchio import casadi as cpin
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

# --- Step 1: Define symbolic variables
L = SX.sym("L", 7)               # Link lengths (X offset)
Y = SX.sym("Y", 7)               # Link offsets in Y direction
theta = SX.sym("theta", 21)      # Orientations: 7 axis-angle vectors (3D each)
q = SX.sym("q", 7)               # Joint configuration

# --- Step 2: Build symbolic model and f_kine with fixed joint types
# Joint types: roll, pitch, yaw, roll, yaw, roll, yaw
joint_axes = [
    SX([1.0, 0.0, 0.0]),  # roll
    SX([0.0, 1.0, 0.0]),  # pitch
    SX([0.0, 0.0, 1.0]),  # yaw
    SX([1.0, 0.0, 0.0]),  # roll
    SX([0.0, 0.0, 1.0]),  # yaw
    SX([1.0, 0.0, 0.0]),  # roll
    SX([0.0, 0.0, 1.0])   # yaw
]

def build_f_kine_fixed():
    model = cpin.Model()
    joint_placement = cpin.SE3.Identity()
    parent_id = 0

    for i in range(7):
        omega = theta[3*i:3*(i+1)]
        R = cpin.exp3(omega)
        p = vertcat(L[i], Y[i], 0)
        T = cpin.SE3(R, p)

        axis_i = joint_axes[i]
        joint_id = model.addJoint(parent_id, cpin.JointModelRevoluteUnaligned(axis_i), joint_placement, f"joint{i}")
        model.appendBodyToJoint(joint_id, cpin.Inertia.Random(), cpin.SE3.Identity())
        model.addFrame(cpin.Frame(f"link{i}", joint_id, 0, cpin.SE3.Identity(), cpin.FrameType.BODY))

        joint_placement = T
        parent_id = joint_id

    data = cpin.Data(model)
    cpin.forwardKinematics(model, data, q)
    p_ee = data.oMi[model.njoints - 1].translation
    return Function("f_kine_fixed", [L, Y, theta, q], [p_ee])

f_kine = build_f_kine_fixed()

# --- Step 3: Define workspace_metric(L, Y, theta) by sampling N q
def build_workspace_metric(N=200):
    q_lower = np.deg2rad([-170, -15, -170, -120, -170, -30, -45])
    q_upper = np.deg2rad([170, 180, 170, 120, 170, 60, 80])
    q_samples = np.random.uniform(q_lower, q_upper, size=(N, 7))
    p_samples = [f_kine(L, Y, theta, q_sample) for q_sample in q_samples]
    mean_p = sum(p_samples) / N
    spread = sum([sum1((p - mean_p)**2) for p in p_samples]) / N
    return Function("workspace_metric", [L, Y, theta], [spread]), q_samples

workspace_metric, q_samples = build_workspace_metric()

# --- Step 4: Optimize workspace_metric(L, Y, theta)
x = vertcat(L, Y, theta)
nlp = {"x": x, "f": -workspace_metric(x[:7], x[7:14], x[14:])}
solver = nlpsol("solver", "ipopt", nlp)

x0 = np.concatenate([np.ones(7) * 0.3, np.zeros(7), np.zeros(21)])
lbx = [0.1]*7 + [-0.1]*7 + [-np.pi]*21
ubx = [0.2]*7 + [0.1]*7 + [np.pi]*21

result = solver(x0=x0, lbx=lbx, ubx=ubx)
x_opt = result["x"].full().flatten()

L_opt = x_opt[:7]
Y_opt = x_opt[7:14]
theta_opt = x_opt[14:]

print("\n✅ Optimized link lengths:", L_opt)
print("✅ Optimized Y offsets:", Y_opt)
print("✅ Optimized orientations (axis-angle):", theta_opt.reshape(7, 3))

# --- Step 5: Visualize optimized robot and workspace in Meshcat (inline notebook mode)
vis = meshcat.Visualizer()
vis.delete()

T = np.eye(4)
def hsv_to_rgb(h, s, v):
    import colorsys
    r, g_, b = colorsys.hsv_to_rgb(h, s, v)
    return int(r * 255) << 16 | int(g_ * 255) << 8 | int(b * 255)
colors = [hsv_to_rgb(i / 7, 0.8, 0.9) for i in range(7)]

for i in range(7):
    name = f"link_{i}"
    length = L_opt[i]
    y_offset = Y_opt[i]
    omega = theta_opt[3*i:3*(i+1)]

    # Box along X
    box_x = g.Box([length, 0.04, 0.04])
    material = g.MeshLambertMaterial(color=colors[i])
    vis[name + "_x"].set_object(box_x, material)
    vis[name + "_x"].set_transform(T @ tf.translation_matrix([length / 2, 0, 0]))

    # Box along Y
    box_y = g.Box([0.04, abs(y_offset), 0.04])
    vis[name + "_y"].set_object(box_y, material)
    vis[name + "_y"].set_transform(T @ tf.translation_matrix([length, y_offset / 2, 0]))

    # Update transform
    R = tf.rotation_matrix(np.linalg.norm(omega)+1e-8, omega / (np.linalg.norm(omega)+1e-8))
    T = T @ tf.translation_matrix([length, y_offset, 0]) @ R

# --- Display sampled workspace points
from scipy.spatial import ConvexHull

workspace_points = []
for i, q_sample in enumerate(q_samples):
    p = f_kine(L_opt, Y_opt, theta_opt, q_sample).full().flatten()
    workspace_points.append(p)
    vis[f"workspace/pt_{i}"].set_object(g.Sphere(0.01), g.MeshLambertMaterial(color=0xff0000))
    vis[f"workspace/pt_{i}"].set_transform(tf.translation_matrix(p))

# --- Convex Hull Display
workspace_points = np.array(workspace_points)
try:
    hull = ConvexHull(workspace_points)
    vertices = workspace_points
    faces = hull.simplices
    vis["workspace/hull"].set_object(
        g.TriangularMeshGeometry(vertices, faces),
        g.MeshLambertMaterial(color=0x00ff00, opacity=0.3, transparent=True)
    )
    print("✅ Convex hull volume:", hull.volume)
except Exception as e:
    print("⚠️ Convex hull failed:", e)

vis.jupyter_cell()


In [ ]:
# 🤖 Symbolic Workspace Estimation with CasADi + Pinocchio.casadi + Optimization + Meshcat Visualisation (Full 3D: L, θ, Fixed Joint Types, Variable Y Offset)

import numpy as np
from casadi import SX, vertcat, Function, sum1, nlpsol
from pinocchio import casadi as cpin
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

# --- Step 1: Define symbolic variables
L = SX.sym("L", 7)               # Link lengths (X offset)
Y = SX.sym("Y", 7)               # Link offsets in Y direction
theta = SX.sym("theta", 21)      # Orientations: 7 axis-angle vectors (3D each)
q = SX.sym("q", 7)               # Joint configuration

# --- Step 2: Build symbolic model and f_kine with fixed joint types
# Joint types: roll, pitch, yaw, roll, yaw, roll, yaw
joint_axes = [
    SX([1.0, 0.0, 0.0]),  # roll
    SX([0.0, 1.0, 0.0]),  # pitch
    SX([0.0, 0.0, 1.0]),  # yaw
    SX([1.0, 0.0, 0.0]),  # roll
    SX([0.0, 0.0, 1.0]),  # yaw
    SX([1.0, 0.0, 0.0]),  # roll
    SX([0.0, 0.0, 1.0])   # yaw
]

def build_f_kine_fixed():
    model = cpin.Model()
    joint_placement = cpin.SE3.Identity()
    parent_id = 0

    for i in range(7):
        omega = theta[3*i:3*(i+1)]
        R = cpin.exp3(omega)
        p = vertcat(L[i], Y[i], 0)
        T = cpin.SE3(R, p)

        axis_i = joint_axes[i]
        joint_id = model.addJoint(parent_id, cpin.JointModelRevoluteUnaligned(axis_i), joint_placement, f"joint{i}")
        model.appendBodyToJoint(joint_id, cpin.Inertia.Random(), cpin.SE3.Identity())
        model.addFrame(cpin.Frame(f"link{i}", joint_id, 0, cpin.SE3.Identity(), cpin.FrameType.BODY))

        joint_placement = T
        parent_id = joint_id

    data = cpin.Data(model)
    cpin.forwardKinematics(model, data, q)
    p_ee = data.oMi[model.njoints - 1].translation
    return Function("f_kine_fixed", [L, Y, theta, q], [p_ee])

f_kine = build_f_kine_fixed()

# --- Step 3: Define workspace_metric(L, Y, theta) by sampling N q

def build_workspace_metric(N=50):
    q_lower = np.deg2rad([-170, -15, -170, -120, -170, -30, -45])
    q_upper = np.deg2rad([170, 180, 170, 120, 170, 60, 80])
    q_samples = np.random.uniform(q_lower, q_upper, size=(N, 7))
    p_samples = [f_kine(L, Y, theta, q_sample) for q_sample in q_samples]
    mean_p = sum(p_samples) / N
    spread = sum([sum1((p - mean_p)**2) for p in p_samples]) / N
    return Function("workspace_metric", [L, Y, theta], [spread]), q_samples

workspace_metric, q_samples = build_workspace_metric()

# --- Step 4: Optimize workspace_metric(L, Y, theta)
x = vertcat(L, Y, theta)
nlp = {"x": x, "f": -workspace_metric(x[:7], x[7:14], x[14:])}
solver = nlpsol("solver", "ipopt", nlp)

x0 = np.concatenate([np.ones(7) * 0.3, np.zeros(7), np.zeros(21)])
lbx = [0.1]*7 + [-0.1]*7 + [-np.pi]*21
ubx = [0.2]*7 + [0.1]*7 + [np.pi]*21

result = solver(x0=x0, lbx=lbx, ubx=ubx)
x_opt = result["x"].full().flatten()

L_opt = x_opt[:7]
Y_opt = x_opt[7:14]
theta_opt = x_opt[14:]

print("\n✅ Optimized link lengths:", L_opt)
print("✅ Optimized Y offsets:", Y_opt)
print("✅ Optimized orientations (axis-angle):", theta_opt.reshape(7, 3))

# --- Step 5A: Build symbolic Jacobian function
# --- Step 5B: Visualize optimized robot and workspace in Meshcat (inline notebook mode)

# Create symbolic Jacobian function
def build_f_jacobian():
    model = cpin.Model()
    joint_placement = cpin.SE3.Identity()
    parent_id = 0
    for i in range(7):
        omega = theta[3*i:3*(i+1)]
        R = cpin.exp3(omega)
        p = vertcat(L[i], Y[i], 0)
        T = cpin.SE3(R, p)
        axis_i = joint_axes[i]
        joint_id = model.addJoint(parent_id, cpin.JointModelRevoluteUnaligned(axis_i), joint_placement, f"joint{i}")
        model.appendBodyToJoint(joint_id, cpin.Inertia.Random(), cpin.SE3.Identity())
        model.addFrame(cpin.Frame(f"link{i}", joint_id, 0, cpin.SE3.Identity(), cpin.FrameType.BODY))
        joint_placement = T
        parent_id = joint_id
    data = cpin.Data(model)
    cpin.forwardKinematics(model, data, q)
    ee_frame_id = model.getFrameId(f"link6")
    J = cpin.computeFrameJacobian(model, data, q, ee_frame_id, cpin.ReferenceFrame.LOCAL_WORLD_ALIGNED)
    return Function("f_jacobian", [L, Y, theta, q], [J])

f_jacobian = build_f_jacobian()

# Display one manipulability ellipsoid in Meshcat
def show_ellipsoid(J, center, name="ellipsoid"):
    import scipy.linalg
    U, s, _ = np.linalg.svd(J[:3, :])
    radii = s / np.max(s)
    ell = g.Ellipsoid(radii)
    # Couleur variant selon manipulabilité
    manipulability = np.prod(s) / np.max(s)**3
    color_val = int(255 * manipulability)
    color_hex = (color_val << 16) | (0x20 << 8) | 0x20
    R = U
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = center
    vis[name].set_object(ell, g.MeshLambertMaterial(color=color_hex, opacity=0.5, transparent=True))
    vis[name].set_transform(T)

vis = meshcat.Visualizer()
vis.open()  # Open external viewer for real-time animation
vis.delete()

T = np.eye(4)
def hsv_to_rgb(h, s, v):
    import colorsys
    r, g_, b = colorsys.hsv_to_rgb(h, s, v)
    return int(r * 255) << 16 | int(g_ * 255) << 8 | int(b * 255)
colors = [hsv_to_rgb(i / 7, 0.8, 0.9) for i in range(7)]

for i in range(7):
    name = f"link_{i}"
    length = L_opt[i]
    y_offset = Y_opt[i]
    omega = theta_opt[3*i:3*(i+1)]

    # Box along X
    box_x = g.Box([length, 0.04, 0.04])
    material = g.MeshLambertMaterial(color=colors[i])
    vis[name + "_x"].set_object(box_x, material)
    vis[name + "_x"].set_transform(T @ tf.translation_matrix([length / 2, 0, 0]))

    # Box along Y
    box_y = g.Box([0.04, abs(y_offset), 0.04])
    vis[name + "_y"].set_object(box_y, material)
    vis[name + "_y"].set_transform(T @ tf.translation_matrix([length, y_offset / 2, 0]))

    # Update transform
    R = tf.rotation_matrix(np.linalg.norm(omega)+1e-8, omega / (np.linalg.norm(omega)+1e-8))
    T = T @ tf.translation_matrix([length, y_offset, 0]) @ R

# --- Display sampled workspace points
from scipy.spatial import ConvexHull

workspace_points = []
for i, q_sample in enumerate(q_samples):
    p = f_kine(L_opt, Y_opt, theta_opt, q_sample).full().flatten()
    workspace_points.append(p)
    vis[f"workspace/pt_{i}"].set_object(g.Sphere(0.01), g.MeshLambertMaterial(color=0xff0000))
    vis[f"workspace/pt_{i}"].set_transform(tf.translation_matrix(p))

# --- Convex Hull Display
workspace_points = np.array(workspace_points)
try:
    hull = ConvexHull(workspace_points)
    vertices = workspace_points
    faces = hull.simplices
    vis["workspace/hull"].set_object(
        g.TriangularMeshGeometry(vertices, faces),
        g.MeshLambertMaterial(color=0x00ff00, opacity=0.3, transparent=True)
    )
    print("✅ Convex hull volume:", hull.volume)
except Exception as e:
    print("⚠️ Convex hull failed:", e)

# --- Animate manipulability ellipsoid along trajectory
import time
from IPython.display import display

q_traj = np.linspace(-0.5, 0.5, 30).reshape(-1, 1) * np.ones((30, 7))  # simple sinusoidal path

def animate():
    for i, qi in enumerate(q_traj):
        J_i = f_jacobian(L_opt, Y_opt, theta_opt, qi).full()
        p_i = f_kine(L_opt, Y_opt, theta_opt, qi).full().flatten()
        show_ellipsoid(J_i, p_i, name="ellipsoid")
        # vis.jupyter_cell()  # Not needed with external viewer
        time.sleep(0.05)

animate()

vis.jupyter_cell()  # Final display omitted with external viewer

# --- DisplayScene utilities

def displayScene(q):
    J = f_jacobian(L_opt, Y_opt, theta_opt, q).full()
    p = f_kine(L_opt, Y_opt, theta_opt, q).full().flatten()
    show_ellipsoid(J, p, name="ellipsoid")

def displayTraj(qs, dt=0.05):
    for q in qs:
        displayScene(q)
        time.sleep(dt)
